## Berechnungen von Merkmalen

In diesem Notebook ist es möglich den Mittelwert, Varianz und Standardabweichung für die aufgenommenen Messwerte aus der Datei "merkmaleRoh.csv" der Android TM Fahrradwege Monitoring App zu berechnen. Zu einem Bild werden parallel Messwerte erfasst von dem Beschleunigungssensor und Magnetsensor des Smartphones, innerhalb eines Zeitfenster mit einer Länge von 80ms (siehe hierzu auch das Notebook "gatheringSequence" in dem Ordner datenvisualisierung). Bei dem Beschleunigungssensor sind dies die Messwerte der beschleunigung von den 3-Achsen X, Y sowie Z (Abb. 1). Des Weiteren werden die Gier-Nick-Roll Winkel in rad in einem Zeitfenster erfasst. Für die Berechnung des Mittelwerts, Varianz und Standardabweichung über die Messwerte wird die Form der abgespeicherten Messwerte geändert, weil diese als Python Objekte verwaltet werden mit welchen nicht gerechnet werden kann. Zu den Messwertreihen wird jeweils eine CSV Datei erstellt. Eine Reihe stellt dann in dieser Datei eine Messreihe zu einem Zeitfenster dar. Bei der ausführung dieses Notebooks werden deshalb zusätzlich folgende CSV Dateien erstellt:

X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2  
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad  
Nick Messwerte in rad  
Roll Messwerte in rad   
Zeitstempel der Messwerte in ns  

<img src="../daten/abbildungen/sensorOrientierung.png" alt="Android Smartphone Sensor Orientierung" width="200px" />
Abbildung 1: 3-Achsen Smartphone Sensor-Koordinatensystem   
Quelle: siehe Bildverzeichnis Abb. 2 in der README.md Datei

In [205]:
import pandas
import numpy as np
import numpy.testing as npt
import math
import os

In [206]:
featuresDf = pandas.read_csv("../daten/merkmale_datensatz_15_bis_22/merkmaleRoh.csv")
featuresDf.describe()

,Zeitstempel in Unixzeit,Anzahl der Messwerte,Belichtungszeit in ns,Breitengrad,Geschwindigkeit in km/h,Hoehe in m ueber WGS 84,Laengengrad,Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM,Start der Belichtung in ns seit Start der JVM,Start der Messwerterfassung in ns seit Start der JVM,Start des Zeitfensters in ns seit Start der JVM,Stopp der Messwerterfassung in Unixzeit
count,1.009970e+05,100997.000000,1.009970e+05,100997.000000,100997.000000,1935.000000,100997.000000,5.116500e+04,1.009970e+05,1.009970e+05,1.009970e+05,5.116500e+04
mean,1.521788e+12,19.369486,1.628067e+07,53.600818,14.825318,70.983613,10.117377,6.609566e+13,7.110740e+13,7.110731e+13,7.110737e+13,1.522338e+12
std,6.253291e+08,0.637294,1.007028e+07,0.026176,3.788162,2.540434,0.064368,6.624152e+13,5.257455e+13,5.257455e+13,5.257455e+13,3.923149e+08
min,1.521124e+12,5.000000,6.435000e+05,53.533375,5.004000,64.299440,9.853853,7.500496e+12,7.500366e+12,7.500275e+12,7.500341e+12,1.521815e+12
25%,1.521212e+12,19.000000,8.963045e+06,53.582508,12.348000,69.145140,10.096094,1.369181e+13,3.100722e+13,3.100713e+13,3.100718e+13,1.521906e+12
50%,1.521815e+12,19.000000,1.000184e+07,53.607080,15.191999,71.069275,10.128578,3.203458e+13,4.411367e+13,4.411357e+13,4.411363e+13,1.522248e+12
75%,1.522248e+12,20.000000,2.999632e+07,53.614410,17.531998,72.942380,10.156504,1.705849e+14,1.087626e+14,1.087624e+14,1.087625e+14,1.522584e+12
max,1.523024e+12,48.000000,3.287369e+07,53.656776,24.948000,77.023680,10.229659,1.762332e+14,1.762331e+14,1.762330e+14,1.762330e+14,1.523024e+12


In [207]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100997 entries, 0 to 100996
Data columns (total 19 columns):
Zeitstempel in Unixzeit                                               100997 non-null int64
Anzahl der Messwerte                                                  100997 non-null int64
Belichtungszeit in ns                                                 100997 non-null int64
Breitengrad                                                           100997 non-null float64
Geschwindigkeit in km/h                                               100997 non-null float64
Gier Messwerte in rad                                                 100997 non-null object
Hoehe in m ueber WGS 84                                               1935 non-null float64
Laengengrad                                                           100997 non-null float64
Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM    51165 non-null float64
Nick Messwerte in rad                                

In [208]:
# Zeigt die ersten 5 Reihen Beschleunigungssensordaten der X-Achse 
featuresDf['X-Achse Beschleunigungswerte in m/s^2'].head()

0    0.66929 0.81137 0.75647 0.22221 -0.14389 -0.13...
1    0.2285 0.13695 0.37016 0.08172 -0.05709 0.3834...
2    -2.72057 3.72297 3.37689 -1.28232 -0.90316 0.4...
3    -0.06289 0.54739 0.6219 -1.35643 -0.54872 0.28...
4    -0.5752 -1.41006 -1.3118 -0.58965 -1.32969 0.6...
Name: X-Achse Beschleunigungswerte in m/s^2, dtype: object

In [209]:
# Die Anzahl der Messwerte sollte immer gleich sein, ansonsten werden fehlende Spalten mit NaN Werten gefüllt bei der
# späteren Abspeicherung der erfassten Messwerte wie z.B. in die Datei x-Achse-Beschleunigungswerte.csv.
# Um die Berechnung von den Merkmalen Varianz und Mittelwert unteranderem zu beschleunigen ist es wichtig sogenannte Außenseiter
# aus den Datensätzen zu entfernen.
# Dafür muss auch das Bild zu einer Reihe aus der merkmaleRoh.csv Datei gelöscht werden.
# Dies wird erreicht indem alle Pfade der Bilder geladen werden.
# Dann können Reihen sowie Bilder gelöscht werden mit mehr Messwerten als ein bestimmter Grenzwert.
print(featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23])

################### Entferne Außenseiter mit Anzahl der Messwerte > Grenzwert ###################
#### Warnung löschen der zugeschnittennen Bilder kann nicht rückgängig gemacht werden!
#threshold  = 23 
#imagePaths = []
# Hier können die Datensätze ausgewählt werden
#datasets = ['15','16','17','18','19','20','21','22']
# Die Pfade zu den Ordnern in welchem sich die Bilder befinden
#paths = []
#for dataset in datasets: # Für jeden Datensatz merke Pfad
#    paths.append("C:/Users/morro/Documents/datenRoh/" + dataset + "/zugeschnitten/")
#for path in paths: # Für jeden Pfad hole die Namen der Ordner
#    folders = os.listdir(path)
#    for folder in folders: # Aus der Liste der Ordner wird ein Ordner ausgewählt
#        filesPath = path + folder + "/"
#        files = os.listdir(filesPath)
#        for name in files: # Ein Dateiname aus diesem Ordner
#            if "jpg" not in name:
#                continue
#            imagePaths.append(filesPath + name) # Path wird geladen in Liste
#toRemoveImagesIndex = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > threshold].index.tolist()
#for i in toRemoveImagesIndex:
#    print(imagePaths[i])
#    os.remove(imagePaths[i])
unwantedRows = featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]
featuresDf.drop(unwantedRows.index, inplace=True)
# Die Inecies müssen zurückgesetzt werden nach den Aufruf von drop. Ansonsten kommte es zu KeyValue Fehlern
featuresDf.reset_index(drop=True,inplace=True)

14959    24
35516    48
35525    28
45468    47
52621    24
54376    24
54719    24
55901    40
56033    24
59055    24
59514    24
59990    38
60418    24
62192    25
64457    28
64506    24
68134    24
80666    24
94258    27
95849    24
97012    43
Name: Anzahl der Messwerte, dtype: int64


In [210]:
featuresDf['Anzahl der Messwerte'][featuresDf['Anzahl der Messwerte'] > 23]

Series([], Name: Anzahl der Messwerte, dtype: int64)

Die folgenden Spalten des Datenframes haben als Type ein Python object.   
X-Achse Beschleunigungswerte in m/s^2   
Y-Achse Beschleunigungswerte in m/s^2   
Z-Achse Beschleunigungswerte in m/s^2   
Gier Messwerte in rad    
Nick Messwerte in rad          
Roll Messwerte in rad                   
Zeitstempel der Messwerte in ns               
Implizit haben diese Spalten als Typ Strings. Aber in einem DataFrame werden Strings als    
Python objects erkannt. Um den Datentyp auf float zu casten wird aus den Spalten zunächst ein Liste erstellt.    
Welche dann wiederum als Numpy Array auf den Typ float konvertiert wird.   

In [211]:
featuresDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100976 entries, 0 to 100975
Data columns (total 19 columns):
Zeitstempel in Unixzeit                                               100976 non-null int64
Anzahl der Messwerte                                                  100976 non-null int64
Belichtungszeit in ns                                                 100976 non-null int64
Breitengrad                                                           100976 non-null float64
Geschwindigkeit in km/h                                               100976 non-null float64
Gier Messwerte in rad                                                 100976 non-null object
Hoehe in m ueber WGS 84                                               1935 non-null float64
Laengengrad                                                           100976 non-null float64
Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM    51148 non-null float64
Nick Messwerte in rad                                

In [212]:
accXList = featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()

In [213]:
# Direkt mit dem Numpy Array ist es nicht möglich die Konvertierung durchzuführen, weil die Arrays von Typ object sind.   
pandas.DataFrame(featuresDf['X-Achse Beschleunigungswerte in m/s^2'].str.split(" ")).values

array([[ ['0.66929', '0.81137', '0.75647', '0.22221', '-0.14389', '-0.13032', '-0.34931', '-0.26401', '-0.349', '-0.41699', '-0.27218', '-0.38619', '-0.29352', '-0.46454', '-0.47877', '-0.3829', '-0.13767', '-0.11002', '0.01936', '0.27609']],
       [ ['0.2285', '0.13695', '0.37016', '0.08172', '-0.05709', '0.38348', '-0.27536', '-0.6339', '0.21317', '-0.05919', '-0.44563', '-0.78543', '-0.52098', '-1.10619', '-0.19531', '-0.7384', '-0.60594', '-0.54593', '0.00775', '-0.26951']],
       [ ['-2.72057', '3.72297', '3.37689', '-1.28232', '-0.90316', '0.47276', '0.60817', '1.13021', '1.93092', '0.62547', '-2.50279', '-0.54644', '0.81943', '-0.67742', '-0.06682', '-0.23722', '0.45391', '1.206', '1.94558', '2.88967']],
       ..., 
       [ ['0.05452', '-0.24745', '-0.44306', '-0.43098', '-0.16084', '-0.18989', '0.10864', '0.05635', '0.45887', '0.81153', '0.83317', '0.89645', '0.67126', '-0.13713', '-0.40077', '-0.19796', '0.34737', '0.43119', '0.45229', '0.10141']],
       [ ['-0.04052', '0

In [214]:
accXList[14958] # zeige Liste

['-0.37509',
 '-1.35734',
 '-0.56488',
 '0.19168',
 '0.39853',
 '-0.89167',
 '-0.60606',
 '0.61842',
 '2.48673',
 '1.34583',
 '-0.34834',
 '0.45684',
 '0.94775',
 '2.03002',
 '0.36756',
 '-1.19226',
 '-1.44413',
 '-1.50771',
 '0.12694']

In [215]:
np.array(accXList[1]).dtype

dtype('<U8')

In [221]:
# Konvertiert die Python Objekte in der Liste zu den Type Float.
# Das Ergebnis wird in einem Dataframe geschrieben
accList = []
accList.append(pandas.DataFrame(abs(np.array(accXList[0]).astype(float))).transpose())
for x in accXList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())

In [222]:
accXDf = pandas.concat(accList, ignore_index=True)

In [223]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0.66929,0.81137,0.75647,0.22221,0.14389,0.13032,0.34931,0.26401,0.34900,0.41699,0.27218,0.38619,0.29352,0.46454,0.47877,0.38290,0.13767,0.11002,0.01936,0.27609,NaN,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,0.05709,0.38348,0.27536,0.63390,0.21317,0.05919,0.44563,0.78543,0.52098,1.10619,0.19531,0.73840,0.60594,0.54593,0.00775,0.26951,NaN,NaN,NaN
2,2.72057,3.72297,3.37689,1.28232,0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,2.50279,0.54644,0.81943,0.67742,0.06682,0.23722,0.45391,1.20600,1.94558,2.88967,NaN,NaN,NaN
3,0.06289,0.54739,0.62190,1.35643,0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,0.58702,1.26652,2.20852,1.35321,1.08268,1.32594,1.07620,2.46997,2.02206,1.61777,1.86127,NaN,NaN
4,0.57520,1.41006,1.31180,0.58965,1.32969,0.65253,1.15038,0.27477,0.61810,0.01130,0.11642,0.23115,1.02780,0.96025,0.44654,0.70976,0.67322,1.87156,1.45116,0.73922,NaN,NaN,NaN


In [225]:
# Um mehr Spalten eines Panda DataFrames auszugeben
pandas.set_option('display.max_columns', 40)
accXDf.T.mean().head(20)

0     0.346705
1     0.383030
2     1.405936
3     1.164702
4     0.807528
5     0.764234
6     0.498969
7     0.738315
8     0.590494
9     1.795119
10    0.580487
11    0.649191
12    1.029057
13    1.593236
14    1.105828
15    1.123956
16    0.568256
17    0.652359
18    0.480085
19    0.510043
dtype: float64

In [226]:
accXDf.shape

(100976, 23)

In [227]:
accYList = featuresDf['Y-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(abs(np.array(accYList[0]).astype(float))).transpose())
for x in accYList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
accYDf = pandas.concat(accList, ignore_index=True)

In [228]:
accYDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100976 entries, 0 to 100975
Data columns (total 23 columns):
0     100976 non-null float64
1     100976 non-null float64
2     100976 non-null float64
3     100976 non-null float64
4     100976 non-null float64
5     100975 non-null float64
6     100972 non-null float64
7     100970 non-null float64
8     100968 non-null float64
9     100966 non-null float64
10    100964 non-null float64
11    100964 non-null float64
12    100959 non-null float64
13    100954 non-null float64
14    100947 non-null float64
15    100939 non-null float64
16    100919 non-null float64
17    100776 non-null float64
18    97361 non-null float64
19    39571 non-null float64
20    1357 non-null float64
21    166 non-null float64
22    40 non-null float64
dtypes: float64(23)
memory usage: 17.7 MB


In [229]:
accYDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0.58300,0.48170,0.38533,1.20873,0.46135,0.75219,0.57112,0.38583,0.09411,0.19785,0.35745,0.10205,0.28613,0.33618,0.34560,0.04666,0.77285,0.71025,0.27709,0.65171,NaN,NaN,NaN
1,0.28650,0.61359,0.29171,0.80856,0.43231,0.85147,0.02369,0.04963,0.45060,0.63627,0.67088,0.50608,0.20570,0.15719,1.13704,0.89429,0.09672,0.36697,0.69194,0.04407,NaN,NaN,NaN
2,0.48539,4.54084,5.10369,3.63862,0.93426,0.06467,1.72190,4.53401,3.90300,2.21832,1.88754,0.39149,0.49890,0.82675,0.69207,0.01738,0.66028,0.34432,0.06090,0.57954,NaN,NaN,NaN
3,0.26624,1.24271,2.05148,3.17350,2.00252,0.17702,0.70112,1.87864,1.05851,1.12260,0.55762,0.66064,1.32553,1.22895,0.07416,0.02870,2.09158,7.03625,8.37177,4.75138,1.77646,NaN,NaN
4,0.74329,4.71651,2.70064,3.55490,2.53744,2.88872,2.51021,2.20739,3.19096,1.89392,1.50345,0.94225,1.64249,1.26800,0.79985,0.72388,1.46786,2.07837,1.67804,2.68746,NaN,NaN,NaN


In [230]:
# Konvertiere Beschleunigungssensordaten Z-Achse zu einem DataFrame
measuredDataList = featuresDf['Z-Achse Beschleunigungswerte in m/s^2'].str.split(" ").tolist()
accList = []
accList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    accList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
accZDf = pandas.concat(accList, ignore_index=True)

In [231]:
# Konvertiere Nick Messwerte zu DataFrame
measuredDataList = featuresDf['Nick Messwerte in rad'].str.split(" ").tolist()
pitchList = []
pitchList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    pitchList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
nickDf = pandas.concat(pitchList, ignore_index=True)

In [233]:
# Konvertiere Roll Messwerte zu DataFrame
measuredDataList = featuresDf['Roll Messwerte in rad'].str.split(" ").tolist()
rollList = []
rollList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    rollList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
rollDf = pandas.concat(rollList, ignore_index=True)

In [234]:
measuredDataList = featuresDf['Gier Messwerte in rad'].str.split(" ").tolist()
azimuthList = []
azimuthList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    azimuthList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
azimuthDf = pandas.concat(azimuthList, ignore_index=True)

In [235]:
measuredDataList = featuresDf['Zeitstempel der Messwerte in ns'].str.split(" ").tolist()
timestampsList = []
timestampsList.append(pandas.DataFrame(abs(np.array(measuredDataList[0]).astype(float))).transpose())
for x in measuredDataList[1:]:
    timestampsList.append(pandas.DataFrame(abs(np.array(x).astype(float))).transpose())
sensorTimestampsDf = pandas.concat(timestampsList, ignore_index=True)

In [236]:
# Neue Spalten werden erstellt und Mittelwerte zugewiesen
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'] = accXDf.T.mean()
featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'] = accYDf.T.mean()
featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'] = accZDf.T.mean()
featuresDf['Mittelwert Nick in rad'] = nickDf.T.mean()
featuresDf['Mittelwert Roll in rad'] = rollDf.T.mean()

In [237]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].head(2)

0    0.346705
1    0.383030
Name: Mittelwert X-Achse Beschleunigung in m/s^2, dtype: float64

In [238]:
featuresDf.columns

Index(['Zeitstempel in Unixzeit', 'Anzahl der Messwerte',
       'Belichtungszeit in ns', 'Breitengrad', 'Geschwindigkeit in km/h',
       'Gier Messwerte in rad', 'Hoehe in m ueber WGS 84', 'Laengengrad',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Start der Belichtung in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2',
       'Zeitstempel der Messwerte in ns',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad'],
      dtype='object')

Berechnet die Varianz. Dieser Funktion muss als Paramter der Mittelwert (mean) und der DataFrame mit   
den Float Werten übergeben werden, um die Varianz zu berechnen. Als Varianz wird der Durchschnitt der quadrierten  
Differenzen zum Mittelwert bezeichnet.   
Prec.:    
Postc.: Gibt die berechnete Variance als Float zurück oder 0 wenn die Anzahl der Werte <= 0  

In [239]:
def calcVariance(meansDf, dfValues):
    variance = []
    for i, values in dfValues.iterrows():
        sum = 0
        for value in values:
            if math.isnan(value) == False:
                tempDifference = value-meansDf[i]
                sum += tempDifference * tempDifference
        variance.append("{0:.5f}".format(round(sum / values.count(),5)))
    return np.array(variance).astype(float)

Hier wird die Standardabweichung der Varianz berechnet. Dies ist die Wurzel der Varianz.   
Für die Berechnung wird der Absolutwert von der Varianz genommen.    
Prec.:
Postc.:  Standardabweichung wird zurückgegeben.

In [240]:
def calculateStandardDeviation(varianceDf):
    deviation = []
    for v in varianceDf:
        temp = math.sqrt(np.abs(v))
        deviation.append("{0:.5f}".format(round(temp,5)))
    return np.array(deviation).astype(float)

Die Funktion berechnet die Winkeldifferenz in rad zwischen den ersten gemessenen Gierwinkel und den letzten Gierwinkel   
innerhalb einer als Argument übergebenen DataFrames. Der zurückgegebene Radiant    
ist immer positiv und gibt die relative änderung in rad an.      
Prec.:   
Postc.: Radiant berechnet   

In [241]:
def calculateAngelChangeAzimuth(azimuthDf):
    result = []
    for i,values in azimuthDf.iterrows():
        first = values[0]*(180/math.pi)
        last = values.iloc[values.count()-1]*(180/math.pi)
        resultTemp = np.abs(first - last)
        if(resultTemp > 180):
            resultTemp = 360 - resultTemp
        result.append("{0:.5f}".format(round(resultTemp * ( math.pi / 180),5)))
    return np.array(result).astype(float)

In [242]:
featuresDf['Gier-Aenderung in rad'] = calculateAngelChangeAzimuth(azimuthDf)

In [243]:
accXDf[14957:14961]

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
14957,0.03740,0.25975,0.19060,0.91865,0.19976,0.69609,0.34235,0.36579,0.59907,1.35266,2.35390,2.74120,1.51872,0.98513,0.02194,0.44215,1.01262,0.18188,1.58587,NaN,NaN,NaN,NaN
14958,0.37509,1.35734,0.56488,0.19168,0.39853,0.89167,0.60606,0.61842,2.48673,1.34583,0.34834,0.45684,0.94775,2.03002,0.36756,1.19226,1.44413,1.50771,0.12694,NaN,NaN,NaN,NaN
14959,1.37906,0.21453,2.25556,0.91573,0.87050,0.57384,1.10265,0.17728,0.80073,1.59062,0.72089,0.43881,0.97931,1.33509,1.19067,0.51844,1.53332,1.21132,0.52467,1.13989,NaN,NaN,NaN
14960,0.97433,1.62225,0.60829,0.77778,0.69553,0.19441,0.29346,0.28620,0.21363,2.17353,1.20253,1.02994,2.27961,0.15347,1.68533,0.01520,1.65803,1.78608,0.91555,NaN,NaN,NaN,NaN


In [244]:
featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'].iloc[14959]

0.97364550000000016

In [245]:
# Berechnung der Varianzen und hinzufügen der Ergebnisse als neue Spalte in dem DataFrame 
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert X-Achse Beschleunigung in m/s^2'],accXDf)
featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Y-Achse Beschleunigung in m/s^2'],accYDf)
featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'] = calcVariance(featuresDf['Mittelwert Z-Achse Beschleunigung in m/s^2'],accZDf)
featuresDf['Varianz Nick in rad'] = calcVariance(featuresDf['Mittelwert Nick in rad'],nickDf)
featuresDf['Varianz Roll in rad'] = calcVariance(featuresDf['Mittelwert Roll in rad'],rollDf)

In [246]:
accXDf.iloc[14959-1]

0     0.37509
1     1.35734
2     0.56488
3     0.19168
4     0.39853
5     0.89167
6     0.60606
7     0.61842
8     2.48673
9     1.34583
10    0.34834
11    0.45684
12    0.94775
13    2.03002
14    0.36756
15    1.19226
16    1.44413
17    1.50771
18    0.12694
19        NaN
20        NaN
21        NaN
22        NaN
Name: 14958, dtype: float64

In [247]:
featuresDf['Varianz X-Achse Beschleunigung in m/s^2'].head()

0    0.04288
1    0.07946
2    1.15662
3    0.40990
4    0.23387
Name: Varianz X-Achse Beschleunigung in m/s^2, dtype: float64

In [248]:
calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])

array([ 0.20707,  0.28189,  1.07546, ...,  0.25687,  0.19927,  0.12506])

In [249]:
# Berechne Standardabweichung
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz X-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Y-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Y-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Z-Achse Beschleunigung in m/s^2'] = calculateStandardDeviation(featuresDf['Varianz Z-Achse Beschleunigung in m/s^2'])
featuresDf['Standardabweichung Nick in rad'] = calculateStandardDeviation(featuresDf['Varianz Nick in rad'])
featuresDf['Standardabweichung Roll in rad'] = calculateStandardDeviation(featuresDf['Varianz Roll in rad'])

In [250]:
featuresDf['Standardabweichung X-Achse Beschleunigung in m/s^2'].head()

0    0.20707
1    0.28189
2    1.07546
3    0.64023
4    0.48360
Name: Standardabweichung X-Achse Beschleunigung in m/s^2, dtype: float64

In [251]:
accXDf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100976 entries, 0 to 100975
Data columns (total 23 columns):
0     100976 non-null float64
1     100976 non-null float64
2     100976 non-null float64
3     100976 non-null float64
4     100976 non-null float64
5     100975 non-null float64
6     100972 non-null float64
7     100970 non-null float64
8     100968 non-null float64
9     100966 non-null float64
10    100964 non-null float64
11    100964 non-null float64
12    100959 non-null float64
13    100954 non-null float64
14    100947 non-null float64
15    100939 non-null float64
16    100919 non-null float64
17    100776 non-null float64
18    97361 non-null float64
19    39571 non-null float64
20    1357 non-null float64
21    166 non-null float64
22    40 non-null float64
dtypes: float64(23)
memory usage: 17.7 MB


In [252]:
accXDf.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22
0,0.66929,0.81137,0.75647,0.22221,0.14389,0.13032,0.34931,0.26401,0.34900,0.41699,0.27218,0.38619,0.29352,0.46454,0.47877,0.38290,0.13767,0.11002,0.01936,0.27609,NaN,NaN,NaN
1,0.22850,0.13695,0.37016,0.08172,0.05709,0.38348,0.27536,0.63390,0.21317,0.05919,0.44563,0.78543,0.52098,1.10619,0.19531,0.73840,0.60594,0.54593,0.00775,0.26951,NaN,NaN,NaN
2,2.72057,3.72297,3.37689,1.28232,0.90316,0.47276,0.60817,1.13021,1.93092,0.62547,2.50279,0.54644,0.81943,0.67742,0.06682,0.23722,0.45391,1.20600,1.94558,2.88967,NaN,NaN,NaN
3,0.06289,0.54739,0.62190,1.35643,0.54872,0.28130,0.57759,0.60009,1.49150,1.49977,0.58702,1.26652,2.20852,1.35321,1.08268,1.32594,1.07620,2.46997,2.02206,1.61777,1.86127,NaN,NaN
4,0.57520,1.41006,1.31180,0.58965,1.32969,0.65253,1.15038,0.27477,0.61810,0.01130,0.11642,0.23115,1.02780,0.96025,0.44654,0.70976,0.67322,1.87156,1.45116,0.73922,NaN,NaN,NaN


In [253]:
# Änderungen in CSV Datei schreiben
#featuresDf.set_index('Zeitstempel in Unixzeit', inplace=True)
featuresDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/merkmale.csv')

In [202]:
pandas.read_csv("../daten/merkmale_datensatz_15_bis_22/merkmale.csv").columns

Index(['Zeitstempel in Unixzeit', 'Anzahl der Messwerte',
       'Belichtungszeit in ns', 'Breitengrad', 'Geschwindigkeit in km/h',
       'Gier Messwerte in rad', 'Hoehe in m ueber WGS 84', 'Laengengrad',
       'Letzter Zeitstempel der Messwerterfassung in ns seit Start der JVM',
       'Nick Messwerte in rad', 'Roll Messwerte in rad',
       'Start der Belichtung in ns seit Start der JVM',
       'Start der Messwerterfassung in ns seit Start der JVM',
       'Start des Zeitfensters in ns seit Start der JVM',
       'Stopp der Messwerterfassung in Unixzeit',
       'X-Achse Beschleunigungswerte in m/s^2',
       'Y-Achse Beschleunigungswerte in m/s^2',
       'Z-Achse Beschleunigungswerte in m/s^2',
       'Zeitstempel der Messwerte in ns',
       'Mittelwert X-Achse Beschleunigung in m/s^2',
       'Mittelwert Y-Achse Beschleunigung in m/s^2',
       'Mittelwert Z-Achse Beschleunigung in m/s^2', 'Mittelwert Nick in rad',
       'Mittelwert Roll in rad', 'Gier-Aenderung in rad',
    

In [254]:
# Speichert die DatenFrames mit den Messwerten in csv Dateien
accXDf.index.names = ["index"]
accXDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/xAchseBeschleunigungswerte.csv')
accYDf.index.names = ["index"]
accYDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/yAchseBeschleunigungswerte.csv')
accZDf.index.names = ["index"]
accZDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/zAchseBeschleunigungswerte.csv')
nickDf.index.names = ["index"]
nickDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/nickMesswerte.csv')
rollDf.index.names = ["index"]
rollDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/rollMesswerte.csv')
azimuthDf.index.names = ["index"]
azimuthDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/azimuthMesswerte.csv')
sensorTimestampsDf.index.names = ["index"]
sensorTimestampsDf.to_csv('../daten/merkmale_datensatz_15_bis_22/absMesswerte/sensoreventZeitstempel.csv')

In [204]:
# Unittests
import unittest

class CalcTest(unittest.TestCase):
    
    
    # Testet die Funktion calcVariance mit einem DataFrame mit den Werten [2.0, 2.0]
    # Der Mittelwert sollte 2.0 sein
    # Das erwartete Ergebnis ist 0.0
    def testCalcVariance2(self):
        meanL = [2.0]
        meanDf = pandas.DataFrame(meanL)
        valuesA = [2.0,2.0]
        valuesDf = pandas.DataFrame([[2.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 0.0,2)
        
    # Testet die Funktion calcVariance mit einem DataFrame mit den folgenden Werten:
    # [1.24, 2.5213, 10.434, 42.45, 5.9]
    # Der Mittelwert ist 12.508
    # Die erwartete Varianz ist 234,247016
    def testCalcVariance5(self):
        meanDf = pandas.DataFrame([12.508])
        valuesDf = pandas.DataFrame([[1.24, 2.52, 10.43, 42.45, 5.9]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf), 234.24702,5)   
    
    # Testet calcVariance mit 0 Datenwerten   
    def testCalcVariance0(self):
        meanDf = pandas.DataFrame([0.0])
        valuesDf = pandas.DataFrame([[0.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),0.0,2)  
        
    # Testet calcVariance mit negativen Datenwerten   
    def testCalcVarianceNegative(self):
        meanDf = pandas.DataFrame([-24.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0]])
        npt.assert_almost_equal(calcVariance(meanDf[0],valuesDf),451.5,1) 
        
    # Testet calcVariance mit Muultidimensionalen DataFrame   
    def testCalcVarianceMultidim(self):
        meanDf = pandas.DataFrame([-24.0,0.0])
        valuesDf = pandas.DataFrame([[-2.0,-24.0,-5.0,7.0],[0.0,0.0,0.0,0.0]])
        # Hier wird Numpy Test verwendet um das Ergebnis (zwei Arrays) zu vergleichen
        npt.assert_array_equal(calcVariance(meanDf[0],valuesDf),np.array([ 451.5,0.0])) 
        
    # Dieser Test testet die Funktion calculateStandardDeviation.
    def testCalculateStandardDeviation(self):
        varianceDf = pandas.DataFrame([451.5])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),21.24853,5) 
        
    # Testet die Funktion calculateStandardDeviation.
    # Dabei hat das Argument den Wert 0.0.
    def testCalculateStandardDeviationZero(self):
        varianceDf = pandas.DataFrame([0.0])
        npt.assert_almost_equal(calculateStandardDeviation(varianceDf[0]),0.0,2)
    
    #  Testet die Funktion calculateAngelChangeAzimuth mit zwei gleichen rad Werten
    def testCalculateAngelChangeAzimuthEqual(self):
        radiants = pandas.DataFrame([[1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),0.0,2)
    
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten -1.0 und 1.0
    def testCalculateAngelChangeAzimuthOne(self):
        radiants = pandas.DataFrame([[-1.0,1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 1.0 und -1.0
    def testCalculateAngelChangeAzimuthOne2(self):
        radiants = pandas.DataFrame([[1.0,-1.0]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),2.0,2)  

    # Testet die Funktion calculateAngelChangeAzimuthValues mit mehreren Werten
    def testCalculateAngelChangeAzimuthValues(self):
        radiants = pandas.DataFrame([[-1.57,1.0,2.0,1.3,1.57]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants),3.14,2)
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde für 
    # die Radianten 0.05 und 0.06
    def testCalculateAngelChangeAzimuthSmall(self):
        radiants = pandas.DataFrame([[0.01,0.02]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 0.01,2)  
        
    # Testet ob die Winkeldifferenz korrekt berechnet wurde mit 
    # negative Radianten 
    def testCalculateAngelChangeAzimuthNegativ(self):
        radiants = pandas.DataFrame([[-2.9,-2.0,-1.28]])
        npt.assert_almost_equal(calculateAngelChangeAzimuth(radiants), 1.62,2) 

unittest.main(argv=[''], verbosity=2, exit=False)

testCalcVariance0 (__main__.CalcTest) ... ok
testCalcVariance2 (__main__.CalcTest) ... ok
testCalcVariance5 (__main__.CalcTest) ... ok
testCalcVarianceMultidim (__main__.CalcTest) ... ok
testCalcVarianceNegative (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthEqual (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthNegativ (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthOne2 (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthSmall (__main__.CalcTest) ... ok
testCalculateAngelChangeAzimuthValues (__main__.CalcTest) ... ok
testCalculateStandardDeviation (__main__.CalcTest) ... ok
testCalculateStandardDeviationZero (__main__.CalcTest) ... ok

----------------------------------------------------------------------
Ran 13 tests in 0.040s

OK
